In [56]:
import csv

csvfile = open("decode.csv")
insts = csv.DictReader(csvfile)
insts = list(filter(lambda x: x["pat"] != "", insts))
for i in insts:
    i["name"] = i["name"].strip()

In [57]:
def fill_zero(inst):
    fields = [
        "invalid",
        "ecall",
        "ebreak",
        "mret",
        "reg_we",
        "branch",
        "jal",
        "jalr",
        "mr",
        "mw",
        "csr",
    ]
    for field in fields:
        inst[field] = int(inst[field] == "1")

In [58]:
def process_alu(inst):
    # alu_sel1/alu_sel2
    alu_sel1_dict = {"rs1": "00", "pc": "01", "zero": "10", "": "??"}
    alu_sel2_dict = {"rs2": "0", "imm": "1", "": "?"}
    inst["alu_sel1"] = alu_sel1_dict[inst["alu_sel1"]]
    inst["alu_sel2"] = alu_sel2_dict[inst["alu_sel2"]]

    # cmp_U
    if inst["cmp_U"] == "":
        inst["cmp_U"] = "?"

    # alu_func
    alu_func = [
        "add",
        "sub",
        "left",
        "right",
        "arith",
        "eq",
        "ne",
        "lt",
        "ge",
        "and",
        "or",
        "xor",
    ]
    alu_func_dict = {"": "????"}
    for i in range(len(alu_func)):
        alu_func_dict[alu_func[i]] = str(bin(i))[2:].zfill(4)
    inst["alu_func"] = alu_func_dict[inst["alu_func"]]

In [59]:
def process_mem(inst):
    # do nothing here
    # mem_len = inst(13,12)
    # load_U  = inst(14)
    pass

In [60]:
def process_wb(inst):
    wb_sel_dict = {"alu": "00", "mem": "01", "snpc": "10", "csr": "11", "": "??"}
    inst["wb_sel"] = wb_sel_dict[inst["wb_sel"]]

In [61]:
def process_inst_type(inst):
    types_dict = {
        "I": "000",
        "U": "001",
        "S": "010",
        "B": "011",
        "J": "100",
        "R": "???",
        "": "???",
    }
    inst["inst_type"] = types_dict[inst["inst_type"]]
    # return inst

In [62]:
funcs = [
    # fill zeros
    fill_zero,
    # alu related signals: [alu_sel1, alu_sel2, alu_func, cmp_U]
    process_alu,
    # mem related signals: []
    process_mem,
    # write-back stage: [wb_sel]
    process_wb,
    # determine inst_type
    process_inst_type,
]
for i in insts:
    for f in funcs:
        f(i)
    for k, v in i.items():
        i[k] = str(v)
# insts = processed

In [63]:
for i in insts:
    print(i)

{'pat': '??????? ????? ????? ??? ????? 01101 11', 'name': 'lui', 'invalid': '0', 'ecall': '0', 'ebreak': '0', 'mret': '0', 'reg_we': '1', 'alu_sel1': '10', 'alu_sel2': '1', 'cmp_U': '?', 'alu_func': '0000', 'branch': '0', 'jal': '0', 'jalr': '0', 'wb_sel': '00', 'inst_type': '001', 'mr': '0', 'mw': '0', 'csr': '0'}
{'pat': '??????? ????? ????? ??? ????? 00101 11', 'name': 'auipc', 'invalid': '0', 'ecall': '0', 'ebreak': '0', 'mret': '0', 'reg_we': '1', 'alu_sel1': '01', 'alu_sel2': '1', 'cmp_U': '?', 'alu_func': '0000', 'branch': '0', 'jal': '0', 'jalr': '0', 'wb_sel': '00', 'inst_type': '001', 'mr': '0', 'mw': '0', 'csr': '0'}
{'pat': '??????? ????? ????? ??? ????? 11011 11', 'name': 'jal', 'invalid': '0', 'ecall': '0', 'ebreak': '0', 'mret': '0', 'reg_we': '1', 'alu_sel1': '01', 'alu_sel2': '1', 'cmp_U': '?', 'alu_func': '0000', 'branch': '0', 'jal': '1', 'jalr': '0', 'wb_sel': '10', 'inst_type': '100', 'mr': '0', 'mw': '0', 'csr': '0'}
{'pat': '??????? ????? ????? ??? ????? 11001 11

In [64]:
class Signal:
    l: int = 0
    r: int = 0
    name: str = ""


offset = 0
signals = {}
for k, v in insts[0].items():
    if k == "pat" or k == "name":
        continue
    signal = Signal()
    signals[k] = signal
    signal.name = k
    signal.l = offset
    signal.r = offset + len(v) - 1
    offset += len(v)
    print(f"val {k} = UInt({len(v)}.W)")
control_len = offset

val invalid = UInt(1.W)
val ecall = UInt(1.W)
val ebreak = UInt(1.W)
val mret = UInt(1.W)
val reg_we = UInt(1.W)
val alu_sel1 = UInt(2.W)
val alu_sel2 = UInt(1.W)
val cmp_U = UInt(1.W)
val alu_func = UInt(4.W)
val branch = UInt(1.W)
val jal = UInt(1.W)
val jalr = UInt(1.W)
val wb_sel = UInt(2.W)
val inst_type = UInt(3.W)
val mr = UInt(1.W)
val mw = UInt(1.W)
val csr = UInt(1.W)


In [65]:
isa = {}
for i in insts:
    Value = str()
    for k, v in i.items():
        if k == "name" or k == "pat":
            continue
        Value += v
    print(f'BitPat("b{i["pat"]}") -> BitPat("b{Value}"),')
    i["control"] = Value
    isa[i["name"]] = {"pat": i["pat"], "ctl": Value}

BitPat("b??????? ????? ????? ??? ????? 01101 11") -> BitPat("b00001101?000000000001000"),
BitPat("b??????? ????? ????? ??? ????? 00101 11") -> BitPat("b00001011?000000000001000"),
BitPat("b??????? ????? ????? ??? ????? 11011 11") -> BitPat("b00001011?000001010100000"),
BitPat("b??????? ????? ????? ??? ????? 11001 11") -> BitPat("b00001001?000000110000000"),
BitPat("b??????? ????? ????? 000 ????? 11000 11") -> BitPat("b0000000000101100??011000"),
BitPat("b??????? ????? ????? 001 ????? 11000 11") -> BitPat("b0000000000110100??011000"),
BitPat("b??????? ????? ????? 100 ????? 11000 11") -> BitPat("b0000000000111100??011000"),
BitPat("b??????? ????? ????? 101 ????? 11000 11") -> BitPat("b0000000001000100??011000"),
BitPat("b??????? ????? ????? 110 ????? 11000 11") -> BitPat("b0000000010111100??011000"),
BitPat("b??????? ????? ????? 111 ????? 11000 11") -> BitPat("b0000000011000100??011000"),
BitPat("b??????? ????? ????? 000 ????? 00000 11") -> BitPat("b00001001?000000001000100"),
BitPat("b?

In [66]:
invalid = ["?" for i in range(control_len)]
invalid[signals["invalid"].l] = "1"
invalid = "".join(invalid)
isa["invalid"] = {"pat": "", "ctl": invalid}
print("b" + isa["invalid"]["ctl"])

b1???????????????????????
